In [1]:
import pandas as pd

In [30]:
index = pd.read_csv('../data/google_data/index.csv')
epi = pd.read_csv('../data/google_data/epidemiology.csv')

In [31]:
territories = ['US', 'US_AS', 'US_GU', 'US_MP', 'US_PR', 'US_VI']
location_keys = index[index.country_code == 'US']['location_key']
# select only us locations and drop the terriroties to leave only states and DC
location_keys = location_keys[[('US' in s and len(s) <= 5) for s in location_keys]][~location_keys.isin(territories)]

In [32]:
# filter all dataframes to only include US states and DC
epi = epi[epi.location_key.isin(location_keys)]

In [33]:
# drop all columns except for date, location_key, and new_confirmed
epi_subset = epi.set_index(['date', 'location_key'])['new_confirmed'].reset_index()

In [39]:
earliest_date = epi_subset[epi_subset["new_confirmed"] > 0]["date"].min()

In [40]:
epi_subset = epi_subset[epi_subset["date"] >= earliest_date]

In [41]:
epi_subset.to_csv('../data/univariate_data.csv', index=False)